In [1]:
from aurora_postgres_helpers import SQLConnection
from transformation_helpers import *

In [2]:
sql = SQLConnection
staging_schema = 'yusra_stories_staging'
production_schema = 'yusra_stories_production'
logs_table = 'logs'

In [3]:
all_logs_df = sql.read_table_to_df(sql, staging_schema, logs_table)
all_logs_df

SQL READ from yusra_stories_staging.logs


,ride_id,log
0,1,--------- beginning of a new ride\n
1,1,2022-10-07 15:08:31.432329 mendoza v9: Getting...
2,1,2022-10-07 15:08:31.932967 mendoza v9: [SYSTEM...
3,1,2022-10-07 15:08:32.433612 mendoza v9: [INFO]:...
4,1,2022-10-07 15:08:32.934279 mendoza v9: [INFO]:...
...,...,...
4167,4,2022-10-07 15:43:18.767880 mendoza v9: [INFO]:...
4168,4,2022-10-07 15:43:19.268598 mendoza v9: [INFO]:...
4169,4,2022-10-07 15:43:19.769327 mendoza v9: [INFO]:...
4170,4,2022-10-07 15:43:20.270015 mendoza v9: [INFO]:...


In [4]:
latest_ride_id = all_logs_df['ride_id'].max()
latest_ride_logs = all_logs_df[(all_logs_df['ride_id'] == latest_ride_id)]

In [5]:
latest_ride_formatted = get_joined_formatted_df(latest_ride_logs)

/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_new_ride'] = df['log'].str.contains('new ride')
/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_info'] = df.log.str.contains('INFO')
/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:168: SettingWithCopyWarning: 
A value is trying to be set on a

In [6]:
latest_user_df = get_users_df(latest_ride_formatted)
latest_user_df

Users dataframe ready to be added to snowflake production schema.


,user_id,name,gender,date_of_birth,age,height_cm,weight_kg,address,email_address,account_created,bike_serial,original_source
3131,4653,Abbie Jones,female,1998-10-11,23.0,188.0,67.0,"561 Freeman underpass,Ahmedhaven,N0 8EL",abbie_j@gmail.com,2021-12-23,SN0000,direct


In [7]:
staging_ride_df = get_staging_rides_df(latest_ride_formatted)
latest_ride_df = get_final_rides_df(staging_ride_df)
latest_ride_df

Rides dataframe ready to be added to snowflake production schema.


/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides_df['total_duration'] = staging_rides_df.groupby(by=['ride_id'], dropna=False)['duration_secs'].transform('max')
/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides_df['total_duration'] = staging_rides_df['total_duration'].apply(lambda x: str(timedelta(seconds=x)))
/Us

,ride_id,user_id,start_time,end_time,total_duration,max_heart_rate_bpm,min_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,total_power_kilojoules
3131,4,4653,2022-10-07 15:34:40,2022-10-07 15:43:21,0:08:40,158,80,111,40,41,7.29


In [8]:
sql.write_df_to_table(sql, latest_user_df, production_schema, 'users', 'append')

Dataframe with 1 ROW(S) APPENDED to users in yusra_stories_production


In [9]:
sql.write_df_to_table(sql, latest_ride_df, production_schema, 'rides', 'append')

Dataframe with 1 ROW(S) APPENDED to rides in yusra_stories_production


In [10]:
sql.drop_table(sql, production_schema, 'users')
sql.drop_table(sql, production_schema, 'rides')

users DROPPED from yusra_stories_production
